# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [2]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
#your code here
spaceship.shape

spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [5]:
valores_perdidos = spaceship.isnull().sum()
print(valores_perdidos)

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64


In [6]:
spaceship_limpio = spaceship.dropna()
print(spaceship_limpio)

     PassengerId HomePlanet CryoSleep     Cabin    Destination   Age    VIP  \
0        0001_01     Europa     False     B/0/P    TRAPPIST-1e  39.0  False   
1        0002_01      Earth     False     F/0/S    TRAPPIST-1e  24.0  False   
2        0003_01     Europa     False     A/0/S    TRAPPIST-1e  58.0   True   
3        0003_02     Europa     False     A/0/S    TRAPPIST-1e  33.0  False   
4        0004_01      Earth     False     F/1/S    TRAPPIST-1e  16.0  False   
...          ...        ...       ...       ...            ...   ...    ...   
8688     9276_01     Europa     False    A/98/P    55 Cancri e  41.0   True   
8689     9278_01      Earth      True  G/1499/S  PSO J318.5-22  18.0  False   
8690     9279_01      Earth     False  G/1500/S    TRAPPIST-1e  26.0  False   
8691     9280_01     Europa     False   E/608/S    55 Cancri e  32.0  False   
8692     9280_02     Europa     False   E/608/S    TRAPPIST-1e  44.0  False   

      RoomService  FoodCourt  ShoppingMall     Spa 

In [9]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
# Assuming spaceship is your DataFrame containing the features and target variable
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")  # Replace with your actual data file

# Separating features and target variable
X = spaceship.drop(columns=['Transported'])
y = spaceship['Transported']

numerical_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
categorical_features = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']

# Define the preprocessing steps for numerical and categorical features, including imputation
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_features),
        
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)

# Define the feature selection step
feature_selection = SelectKBest(score_func=f_classif, k=10)

# Create a pipeline that combines the preprocessing and feature selection
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selection', feature_selection)
])

# Fit the pipeline to the data
X_preprocessed = pipeline.fit_transform(X, y)

# Display the selected feature names after one-hot encoding and scaling
selected_features = pipeline.named_steps['feature_selection'].get_support(indices=True)
feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
selected_feature_names = feature_names[selected_features]

print("Selected Features:")
print(selected_feature_names)

Selected Features:
['num__Age' 'num__RoomService' 'num__Spa' 'num__VRDeck'
 'cat__HomePlanet_Earth' 'cat__HomePlanet_Europa' 'cat__CryoSleep_False'
 'cat__CryoSleep_True' 'cat__Destination_55 Cancri e'
 'cat__Destination_TRAPPIST-1e']


In [11]:
#your code here
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
# Assuming spaceship is your DataFrame containing the features and target variable
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")  # Replace with your actual data file

# Separating features and target variable
X = spaceship.drop(columns=['Transported'])
y = spaceship['Transported']

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identifying numerical and categorical features
numerical_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
categorical_features = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']

# Define the preprocessing steps for numerical and categorical features, including imputation
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_features),
        
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)

# Define the feature selection step
feature_selection = SelectKBest(score_func=f_classif, k=10)

# Create a pipeline that combines the preprocessing and feature selection
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selection', feature_selection)
])
# Fit the pipeline to the training data and transform both train and test sets
X_train_preprocessed = pipeline.fit_transform(X_train, y_train)
X_test_preprocessed = pipeline.transform(X_test)

# Display the selected feature names after one-hot encoding and scaling
selected_features = pipeline.named_steps['feature_selection'].get_support(indices=True)
feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
selected_feature_names = feature_names[selected_features]

print("Selected Features:")
print(selected_feature_names)

Selected Features:
['num__Age' 'num__RoomService' 'num__Spa' 'num__VRDeck'
 'cat__HomePlanet_Earth' 'cat__HomePlanet_Europa' 'cat__CryoSleep_False'
 'cat__CryoSleep_True' 'cat__Destination_55 Cancri e'
 'cat__Destination_TRAPPIST-1e']


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [13]:
#your code here
#your code here

import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import BaggingClassifier 
from sklearn.metrics import accuracy_score

# Assuming spaceship is your DataFrame containing the features and target variable
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")  

X = spaceship.drop(columns=['Transported'])
y = spaceship['Transported']

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing pipeline (as defined earlier)
numerical_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
categorical_features = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_features),
        
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)

feature_selection = SelectKBest(score_func=f_classif, k=10)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selection', feature_selection)
])

# Fit and transform the training data
X_train_preprocessed = pipeline.fit_transform(X_train, y_train)
X_test_preprocessed = pipeline.transform(X_test)

# Define the base estimator
base_estimator = DecisionTreeClassifier(random_state=42)

# Bagging Classifier
bagging_clf = BaggingClassifier(
    estimator=base_estimator,  # Updated argument name
    n_estimators=50,           # Number of base models
    bootstrap=True,            # Enables bootstrapping for Bagging
    random_state=42,
    n_jobs=-1                  # Use all available CPU cores
)
bagging_clf.fit(X_train_preprocessed, y_train)
y_pred_bagging = bagging_clf.predict(X_test_preprocessed)
bagging_accuracy = accuracy_score(y_test, y_pred_bagging)

# Pasting Classifier
pasting_clf = BaggingClassifier(
    estimator=base_estimator,  # Updated argument name
    n_estimators=50,           # Number of base models
    bootstrap=False,           # Disables bootstrapping for Pasting
    random_state=42,
    n_jobs=-1                  # Use all available CPU cores
)
pasting_clf.fit(X_train_preprocessed, y_train)
y_pred_pasting = pasting_clf.predict(X_test_preprocessed)
pasting_accuracy = accuracy_score(y_test, y_pred_pasting)

print(f"Bagging Accuracy: {bagging_accuracy:.4f}")
print(f"Pasting Accuracy: {pasting_accuracy:.4f}")


Bagging Accuracy: 0.7573
Pasting Accuracy: 0.7136


- Evaluate your model

In [14]:
#your code here
#your code here

import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import BaggingClassifier 
from sklearn.metrics import accuracy_score

# Assuming spaceship is your DataFrame containing the features and target variable
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")  

X = spaceship.drop(columns=['Transported'])
y = spaceship['Transported']

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing pipeline (as defined earlier)
numerical_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
categorical_features = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_features),
        
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)

feature_selection = SelectKBest(score_func=f_classif, k=10)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selection', feature_selection)
])

# Fit and transform the training data
X_train_preprocessed = pipeline.fit_transform(X_train, y_train)
X_test_preprocessed = pipeline.transform(X_test)

# Define the base estimator
base_estimator = DecisionTreeClassifier(random_state=42)

# Bagging Classifier
bagging_clf = BaggingClassifier(
    estimator=base_estimator,  # Updated argument name
    n_estimators=50,           # Number of base models
    bootstrap=True,            # Enables bootstrapping for Bagging
    random_state=42,
    n_jobs=-1                  # Use all available CPU cores
)
bagging_clf.fit(X_train_preprocessed, y_train)
y_pred_bagging = bagging_clf.predict(X_test_preprocessed)
bagging_accuracy = accuracy_score(y_test, y_pred_bagging)

# Pasting Classifier
pasting_clf = BaggingClassifier(
    estimator=base_estimator,  # Updated argument name
    n_estimators=50,           # Number of base models
    bootstrap=False,           # Disables bootstrapping for Pasting
    random_state=42,
    n_jobs=-1                  # Use all available CPU cores
)
pasting_clf.fit(X_train_preprocessed, y_train)
y_pred_pasting = pasting_clf.predict(X_test_preprocessed)
pasting_accuracy = accuracy_score(y_test, y_pred_pasting)

print(f"Bagging Accuracy: {bagging_accuracy:.4f}")
print(f"Pasting Accuracy: {pasting_accuracy:.4f}")

Bagging Accuracy: 0.7573
Pasting Accuracy: 0.7136


In [15]:
#your code here
from sklearn.ensemble import RandomForestClassifier

# Define the Random Forest Classifier
random_forest_clf = RandomForestClassifier(
    n_estimators=100,   # Number of trees in the forest
    random_state=42,
    n_jobs=-1           # Use all available CPU cores
)

# Fit the Random Forest model to the training data
random_forest_clf.fit(X_train_preprocessed, y_train)

# Predict the target variable using the test set
y_pred_rf = random_forest_clf.predict(X_test_preprocessed)

# Calculate accuracy for the Random Forest model
rf_accuracy = accuracy_score(y_test, y_pred_rf)

print(f"Random Forest Accuracy: {rf_accuracy:.4f}")


Random Forest Accuracy: 0.7522


In [16]:
#your code here
from sklearn.ensemble import GradientBoostingClassifier

# Define the Gradient Boosting Classifier
gradient_boosting_clf = GradientBoostingClassifier(
    n_estimators=100,      # Number of boosting stages to perform
    learning_rate=0.1,     # Step size shrinkage
    max_depth=3,           # Maximum depth of individual trees
    random_state=42
)

# Fit the Gradient Boosting model to the training data
gradient_boosting_clf.fit(X_train_preprocessed, y_train)

# Predict the target variable using the test set
y_pred_gb = gradient_boosting_clf.predict(X_test_preprocessed)

# Calculate accuracy for the Gradient Boosting model
gb_accuracy = accuracy_score(y_test, y_pred_gb)

print(f"Gradient Boosting Accuracy: {gb_accuracy:.4f}")


Gradient Boosting Accuracy: 0.7769


In [17]:
#your code here
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Define the AdaBoost Classifier without a base_estimator argument
adaboost_clf = AdaBoostClassifier(
    n_estimators=50,      # Number of weak learners
    learning_rate=1.0,    # Weighting of each weak learner
    random_state=42
)

# Fit the AdaBoost model to the training data
adaboost_clf.fit(X_train_preprocessed, y_train)

# Predict the target variable using the test set
y_pred_ada = adaboost_clf.predict(X_test_preprocessed)

# Calculate accuracy for the AdaBoost model
ada_accuracy = accuracy_score(y_test, y_pred_ada)

print(f"AdaBoost Accuracy: {ada_accuracy:.4f}")

AdaBoost Accuracy: 0.7625


c:\Users\Casa\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [18]:
#your code here
param_grid = {
    'n_estimators': [50, 100, 200],         # Number of boosting rounds
    'learning_rate': [0.01, 0.05, 0.1, 0.2], # Step size
    'max_depth': [3, 4, 5, 6],              # Depth of each tree
    'min_samples_split': [2, 5, 10],        # Minimum samples for splitting a node
    'min_samples_leaf': [1, 2, 4],          # Minimum samples in a leaf
    'subsample': [0.8, 0.9, 1.0]            # Fraction of samples used per tree
}

- Run Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Define the base model
gb_clf = GradientBoostingClassifier(random_state=42)
# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=gb_clf, 
    param_grid=param_grid,
    cv=5,              # 5-fold cross-validation
    n_jobs=-1,         # Use all available cores
    scoring='accuracy',
    verbose=2
)

# Fit GridSearchCV to the training data
grid_search.fit(X_train_preprocessed, y_train)

# Extract the best model from GridSearchCV
best_gb_clf = grid_search.best_estimator_

# Predict using the best model
y_pred_best_gb = best_gb_clf.predict(X_test_preprocessed)

# Evaluate the best model's accuracy
best_gb_accuracy = accuracy_score(y_test, y_pred_best_gb)

print(f"Best Gradient Boosting Accuracy after tuning: {best_gb_accuracy:.4f}")
print(f"Best Hyperparameters: {grid_search.best_params_}")

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


- Evaluate your model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define the base model
rf_clf = RandomForestClassifier(random_state=42)

# Define the hyperparameter grid for tuning
param_grid = {
    'n_estimators': [50, 100, 200, 300],     # Number of trees
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider at each split
    'max_depth': [10, 20, 30, None],         # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],         # Minimum samples for splitting a node
    'min_samples_leaf': [1, 2, 4],           # Minimum samples in a leaf
    'bootstrap': [True, False]               # Whether to use bootstrapping
}

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=rf_clf, 
    param_grid=param_grid,
    cv=5,              # 5-fold cross-validation
    n_jobs=-1,         # Use all available cores
    scoring='accuracy',
    verbose=2
)

# Fit GridSearchCV to the training data
grid_search.fit(X_train_preprocessed, y_train)

# Extract the best model from GridSearchCV
best_rf_clf = grid_search.best_estimator_

# Predict using the best model
y_pred_best_rf = best_rf_clf.predict(X_test_preprocessed)

# Evaluate the best model's accuracy
best_rf_accuracy = accuracy_score(y_test, y_pred_best_rf)

print(f"Best Random Forest Accuracy after tuning: {best_rf_accuracy:.4f}")
print(f"Best Hyperparameters: {grid_search.best_params_}")
